In [ ]:
### Examine Large Baskets
# A basket is a tuple (ID, Session, Scan, Date)

for df in df_all:
    for pid in df.ID.unique():
        for session in df.loc[df.ID == pid, 'Session'].unique():
            for scan in df.loc[(df.ID == pid) & (df.Session == session), 'Scan'].unique():
                for date in df.loc[(df.ID == pid) & (df.Session == session) & (df.Scan == scan), 'Date'].unique():
                    if df[(df.ID == pid) & (df.Session == session) & (df.Scan == scan) & (df.Date == date)].shape[0] >= 40:
                        print(f'({pid}, {session}, {scan}):', [str(d) for d in df.loc[(df.ID == pid) & (df.Session == session) & (df.Scan == scan), 'Date'].unique()])
    print()

In [ ]:
## Examine Missing Scan / ReceiptNum
for df in df_all:
    for pid in df.ID.unique():
        for session in df.loc[df.ID == pid, 'Session'].unique():
            for date in df.loc[(df.ID == pid) & (df.Session == session) & (df.Scan == -1), 'Date'].unique():
                print(f'({pid}, {session}, -1):', [str(d) for d in df.loc[(df.ID == pid) & (df.Session == session) & (df.Scan == -1), 'Date'].unique()])
    print()

In [ ]:
## Scan / ReceiptNum with multiple dates
for df in df_all:
    for pid in df.ID.unique():
        for session in df.loc[df.ID == pid, 'Session'].unique():
            for scan in df.loc[(df.ID == pid) & (df.Session == session), 'Scan'].unique():
                if (len(df.loc[(df.ID == pid) & (df.Session == session) & (df.Scan == scan), 'Date'].unique()) > 1) and (scan != -1):
                    print(f'({pid}, {session}, {scan}):', [str(d) for d in df.loc[(df.ID == pid) & (df.Session == session) & (df.Scan == scan), 'Date'].unique()])
    print()